In [547]:
import pandas as pd 
import numpy as np
import os

In [548]:
print(os.getcwd())

/home/taoyida/QS4ML-VU-100/final_data


In [549]:
raw_merge_data_dir = os.getcwd() + '/raw_merge_data/'
raw_merge_data_files = ['walk_merge_raw.csv', 'bike_merge_raw.csv', 'run_merge_raw.csv', 'sit_merge_raw.csv', 'syn_merge_raw.csv']

#所有dataframe都存在这里
df_list = []

for i in range(len(raw_merge_data_files)):
    df = pd.read_csv(raw_merge_data_dir + raw_merge_data_files[i], low_memory=False)
    df_list.append(df)

清洗数据

In [550]:
# Hyperparameters
# Define thresholds for specific columns
thresholds = {
    "latitude": (-90, 90),
    "longitude": (-180, 180),
    "altitude": (-500, 12000),
    "course": (0, 360),
    "hacc": (0, 100),  # Assuming max horizontal accuracy of 100 meters
    "speed": (0, 300),  # Assuming max speed of 300 m/s
}
XYZ_columns = ["dateTime", "X", "Y", "Z", "Type"]
relevant_columns = [
    "dateTime",
    "X",
    "Y",
    "Z",
    "Type",
    "altitude",
    "course",
    "hacc",
    "latitude",
    "longitude",
    "speed",
]
columns_to_check = [
    "X",
    "Y",
    "Z",
    "altitude",
    "course",
    "hacc",
    "latitude",
    "longitude",
    "speed",
]
geolocation_columns = ["altitude", "course", "latitude", "longitude"]

In [551]:
def remove_outliers_and_apply_thresholds(df, columns,thresholds):
    for col in columns:
        if col in thresholds:
            df = df[(df[col] >= thresholds[col][0]) & (df[col] <= thresholds[col][1])]
        else:
            df = df[(df[col] >= df[col].min()) & (df[col] <= df[col].max())]
    return df

In [552]:
def extract_XYZcolumns(df):
    return df[["dateTime", "X", "Y", "Z", "Type"]].copy()


def extract_geolocation_columns(df):
    return pd.concat([df["dateTime"], df[geolocation_columns]], axis=1).copy()


def extract_remaining_columns(df):
    return df[["dateTime","hacc", "speed"]].copy()

XYZ变量命名规则：变量名_X

In [553]:
def XYZsplit(df):
    # Drop rows where 'Type' is NaN
    df = df.dropna(subset=['Type']).copy()

    # Create columns for each type
    types = df["Type"].unique()
    for t in types:
        for col in ["X", "Y", "Z"]:
            df.loc[:, f"{t}_{col}"] = df.apply(
                lambda row: row[col] if row["Type"] == t else None, axis=1
            )

    # Explicitly list columns to retain, no need for Position columns
    columns_to_keep = ["dateTime"] + [
        f"{t}_{col}"
        for t in types
        for col in ["X", "Y", "Z"]
        if f"{t}_{col}" in df.columns
    ]
    df = df[columns_to_keep]
    # Round the numeric columns to the desired decimal places
    # Uncomment and modify the line below to set the desired decimal places
    df = df.round(6)

    return df

In [554]:
test=df_list[0]

In [555]:
test.describe()

,BandAccX,BandAccY,BandAccZ,X,Y,Z,altitude,course,hacc,latitude,longitude,speed,rate,rateZone
count,2126.000000,2126.000000,2126.000000,30099.000000,30099.000000,30099.000000,0.0,0.0,0.0,0.0,0.0,0.0,88.000000,88.000000
mean,3605.002226,1805.011312,362.367654,2.080908,-4.754735,-3.582044,NaN,NaN,NaN,NaN,NaN,NaN,118.647727,0.619773
std,1860.071295,950.985917,675.714709,28.818155,24.109581,27.370891,NaN,NaN,NaN,NaN,NaN,NaN,15.768468,0.083059
min,-4481.400000,-1898.200000,-2124.200000,-179.864453,-89.637921,-179.962289,NaN,NaN,NaN,NaN,NaN,NaN,80.000000,0.420000
25%,3379.250000,1151.275000,26.658333,-1.289133,-14.242152,-3.766303,NaN,NaN,NaN,NaN,NaN,NaN,112.000000,0.580000
50%,4070.500000,1641.000000,386.533333,0.681356,5.531010,1.079761,NaN,NaN,NaN,NaN,NaN,NaN,120.000000,0.630000
75%,4685.500000,2308.200000,668.100000,3.590430,8.864810,6.930213,NaN,NaN,NaN,NaN,NaN,NaN,131.000000,0.680000
max,8269.000000,7045.400000,6013.800000,179.533886,88.752277,179.929709,NaN,NaN,NaN,NaN,NaN,NaN,142.000000,0.740000


In [556]:
# Convert dateTime to pd.datetime
def convert_to_datetime(df):
    df['dateTime'] = pd.to_datetime(df['dateTime'])
    #Create a new column for the second
    df['Second'] = df['dateTime'].dt.floor('S')
    return df

思路：将总数据表拆分为XYZ数据，相同秒内前向插值；和地理数据，线性插值；和其他数据（手环和手机），保持时间戳以横向合并

In [557]:
data = convert_to_datetime(test)
XYZdata=extract_XYZcolumns(data)
geodata=extract_geolocation_columns(data)
otherdata=extract_remaining_columns(data)

mat数据由：XYZ数据，地理数据和剩余数据以及时间戳构成。上述所有的data均有时间戳做主元。清洗方法给在下面

In [558]:
XYZdata_splited = XYZsplit(XYZdata.copy())
# Set 'dateTime' as the index
XYZdata_splited.set_index('dateTime', inplace=True)

# Group by index (dateTime) and calculate the mean of all columns in each group
XYZdata_splited = XYZdata_splited.groupby('dateTime').mean()

# Reset the index
XYZdata_splited.reset_index(inplace=True)
XYZdata_splited.head()

,dateTime,MagneticField_X,MagneticField_Y,MagneticField_Z,Acceleration_X,Acceleration_Y,Acceleration_Z,Orientation_X,Orientation_Y,Orientation_Z,AngularVelocity_X,AngularVelocity_Y,AngularVelocity_Z
0,2024-06-06 11:48:12.536,-6.03750,-10.06875,-45.375000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-06-06 11:48:12.539,NaN,NaN,NaN,4.573282,-0.102891,8.361719,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-06-06 11:48:12.559,NaN,NaN,NaN,4.464409,-0.059821,8.483752,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-06 11:48:12.579,NaN,NaN,NaN,4.690531,0.062213,8.280363,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-06-06 11:48:12.596,-6.46875,-10.21875,-45.712502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [559]:
from scipy.spatial.transform import Rotation as R
from scipy.spatial.transform import Slerp


def linear_columns_interpolation(df, columns):
    for col in columns:
        df[col] = df[col].interpolate(
            method="linear", limit_direction="both", limit_area="inside"
        )
    return df


def b_and_ffill_columns_interpolation(df, columns):
    for col in columns:
        df[col] = df[col].bfill().ffill()
    return df


def limited_columns_interpolation(df, columns):
    for col in columns:
        # 对整个列进行插值
        df[col] = df[col].interpolate(method="linear", limit_direction="both", limit=5)
        # 对仍然为 NaN 的部分设为 0
        df[col] = df[col].fillna(0)
    return df


def interpolate_orientation(df, columns):
    # Check if the dataframe has the required columns
    length = len(df)
    if not all(col in df.columns for col in columns):
        raise ValueError("DataFrame does not contain all required columns")
    
    # Convert the orientation columns to Rotation objects, handling NaNs
    indices = []
    rotations = []
    for index, row in df[columns].iterrows():
        if not row.isnull().any():
            indices.append(index)
            rotations.append(R.from_euler('xyz', row, degrees=True))
    
    # Check if there are at least two valid data points to perform interpolation
    if len(indices) < 2:
        raise ValueError("Not enough valid data points to perform interpolation")
    
    slerp = Slerp(indices, R.from_quat([r.as_quat() for r in rotations]))
    
    for i in range(len(df)):
        if df.iloc[i][columns].isnull().any():
            # Ensure the interpolation index is within the valid range
            if i >= indices[0] and i <= indices[-1]:
                df.loc[i, columns] = slerp([i])[0].as_euler('xyz', degrees=True)
    
    # Fill any remaining NaNs (if interpolation limit is reached) with zeros or other strategy
    df[columns] = df[columns].fillna(0)
    df=df[:length]
    
    return df


全部线性插值列：MagneticField_X	MagneticField_Y	MagneticField_Z
有限线性插值列： Acceleration_X	Acceleration_Y	Acceleration_Z AngularVelocity_X	AngularVelocity_Y	AngularVelocity_Z
方位角特殊插值： Orientation_X	Orientation_Y	Orientation_Z 

In [560]:
t = XYZdata_splited.copy()
t = linear_columns_interpolation(t, t.columns[1:4])

In [561]:
t=limited_columns_interpolation(t, t.columns[4:7])

In [562]:
t = interpolate_orientation(t, t.columns[7:10])

In [563]:
t=limited_columns_interpolation(t, t.columns[10:])

In [564]:
t.describe()

,dateTime,MagneticField_X,MagneticField_Y,MagneticField_Z,Acceleration_X,Acceleration_Y,Acceleration_Z,Orientation_X,Orientation_Y,Orientation_Z,AngularVelocity_X,AngularVelocity_Y,AngularVelocity_Z
count,29278,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000
mean,2024-06-06 11:51:39.773743616,-0.145978,-23.778468,-12.138202,0.767981,6.272622,2.635611,23.049010,-50.018902,-39.497884,0.000799,0.000953,0.000023
min,2024-06-06 11:48:12.536000,-44.156250,-66.337502,-49.181252,-18.704695,-42.055054,-39.928432,-179.983681,-89.977197,-179.999185,-0.780572,-1.933587,-2.590588
25%,2024-06-06 11:49:54.022999808,-15.731251,-37.818752,-30.225000,-0.683076,4.951648,-0.263809,-67.016277,-80.480619,-129.354520,0.000000,0.000000,0.000000
50%,2024-06-06 11:51:41.026999808,3.712500,-30.206251,-13.305470,0.591625,7.042675,2.523830,35.014449,-60.943905,-9.097937,0.000000,0.000000,0.000000
75%,2024-06-06 11:53:22.544000,15.168751,-15.787500,6.251953,2.203790,9.729890,7.016878,111.800912,-36.857772,2.988578,0.000000,0.000000,0.000000
max,2024-06-06 11:55:12.708000,38.193752,54.037502,45.956253,19.944775,42.606003,28.344778,179.987222,89.789817,179.996782,1.401754,2.053029,1.068992
std,NaN,17.676106,23.479777,21.060580,2.828916,5.996126,5.696906,100.748961,42.245068,86.380411,0.067590,0.094919,0.086854


In [565]:
geodata.describe()

,dateTime,altitude,course,latitude,longitude
count,32190,0.0,0.0,0.0,0.0
mean,2024-06-06 11:51:42.871313664,NaN,NaN,NaN,NaN
min,2024-06-06 11:48:12.529000,NaN,NaN,NaN,NaN
25%,2024-06-06 11:49:56.154999808,NaN,NaN,NaN,NaN
50%,2024-06-06 11:51:45.210999808,NaN,NaN,NaN,NaN
75%,2024-06-06 11:53:28.864249856,NaN,NaN,NaN,NaN
max,2024-06-06 11:55:12.715000,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN


In [566]:
u = geodata.copy()
u = linear_columns_interpolation(u, u.columns[1:])

In [567]:
otherdata.describe()

,dateTime,hacc,speed
count,32190,0.0,0.0
mean,2024-06-06 11:51:42.871313664,NaN,NaN
min,2024-06-06 11:48:12.529000,NaN,NaN
25%,2024-06-06 11:49:56.154999808,NaN,NaN
50%,2024-06-06 11:51:45.210999808,NaN,NaN
75%,2024-06-06 11:53:28.864249856,NaN,NaN
max,2024-06-06 11:55:12.715000,NaN,NaN
std,NaN,NaN,NaN


In [568]:
y=otherdata.copy()
y=b_and_ffill_columns_interpolation(y, y.columns)

In [569]:
t['dateTime'].value_counts()

dateTime
2024-06-06 11:48:12.536    1
2024-06-06 11:52:48.809    1
2024-06-06 11:52:48.772    1
2024-06-06 11:52:48.769    1
2024-06-06 11:52:48.749    1
                          ..
2024-06-06 11:50:27.776    1
2024-06-06 11:50:27.773    1
2024-06-06 11:50:27.756    1
2024-06-06 11:50:27.736    1
2024-06-06 11:55:12.708    1
Name: count, Length: 29278, dtype: int64

In [570]:
merged_df = pd.merge(t, y, on='dateTime', how='inner')
merged_df = pd.merge(merged_df, u, on='dateTime', how='inner')
merged_df.drop_duplicates(subset=['dateTime'], inplace=True)


In [571]:
merged_df.describe()

,dateTime,MagneticField_X,MagneticField_Y,MagneticField_Z,Acceleration_X,Acceleration_Y,Acceleration_Z,Orientation_X,Orientation_Y,Orientation_Z,AngularVelocity_X,AngularVelocity_Y,AngularVelocity_Z,hacc,speed,altitude,course,latitude,longitude
count,29278,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,29278.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,2024-06-06 11:51:39.773743616,-0.145978,-23.778468,-12.138202,0.767981,6.272622,2.635611,23.049010,-50.018902,-39.497884,0.000799,0.000953,0.000023,NaN,NaN,NaN,NaN,NaN,NaN
min,2024-06-06 11:48:12.536000,-44.156250,-66.337502,-49.181252,-18.704695,-42.055054,-39.928432,-179.983681,-89.977197,-179.999185,-0.780572,-1.933587,-2.590588,NaN,NaN,NaN,NaN,NaN,NaN
25%,2024-06-06 11:49:54.022999808,-15.731251,-37.818752,-30.225000,-0.683076,4.951648,-0.263809,-67.016277,-80.480619,-129.354520,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,2024-06-06 11:51:41.026999808,3.712500,-30.206251,-13.305470,0.591625,7.042675,2.523830,35.014449,-60.943905,-9.097937,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,2024-06-06 11:53:22.544000,15.168751,-15.787500,6.251953,2.203790,9.729890,7.016878,111.800912,-36.857772,2.988578,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
max,2024-06-06 11:55:12.708000,38.193752,54.037502,45.956253,19.944775,42.606003,28.344778,179.987222,89.789817,179.996782,1.401754,2.053029,1.068992,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,17.676106,23.479777,21.060580,2.828916,5.996126,5.696906,100.748961,42.245068,86.380411,0.067590,0.094919,0.086854,NaN,NaN,NaN,NaN,NaN,NaN
